In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
files = ['1.csv', '2.csv', '3.csv', '4.csv', '5.csv', '6.csv']

In [3]:
#reading all the files into dataframes
df1 = pd.read_csv('1.csv')
df2 = pd.read_csv('2.csv')
df3 = pd.read_csv('3.csv')
df4 = pd.read_csv('4.csv')
df5 = pd.read_csv('5.csv')
df6 = pd.read_csv('6.csv')

In [4]:
dfs = [df1, df2, df3, df4, df5, df6]

data = pd.concat(dfs, axis=0, ignore_index=True)

In [5]:
data = data.iloc[:-5,:]

data = data.rename(columns={"Series Name": "series_name", "Series Code": "series_code", "Country Name":"country_name", "Country Code":"country_code",
                                  "2003 [YR2003]" : "2003", "2004 [YR2004]": "2004" ,"2005 [YR2005]": "2005", "2006 [YR2006]": "2006",
                                  "2007 [YR2007]" : "2007", "2008 [YR2008]": "2008", "2009 [YR2009]": "2009", "2010 [YR2010]": "2010", 
                                  "2011 [YR2011]" : "2011", "2012 [YR2012]": "2012", "2013 [YR2013]": "2013", "2014 [YR2014]": "2014", 
                                  "2015 [YR2015]" : "2015", "2016 [YR2016]": "2016", "2017 [YR2017]": "2017", "2018 [YR2018]": "2018", 
                                  "2019 [YR2019]" : "2019", "2020 [YR2020]": "2020", "2021 [YR2021]": "2021", "2022 [YR2022]": "2022"})
data = data.drop("series_code", axis=1)


In [6]:
# move year columns into one
df_pivot = data.melt(id_vars=["series_name","country_name", "country_code"], 
        var_name="year", 
        value_name="value")

# move rows into columns
gender_df = pd.pivot_table(df_pivot, values='value', index=['country_name', "country_code", "year"], columns=['series_name'], aggfunc="sum")

# remove multi_index
gender_df.columns = [col for col in gender_df.columns]
gender_df.reset_index(inplace=True)

# replace all .. with NAN
gender_df = gender_df.replace(-1, np.nan)

In [7]:
gender_df.shape


# Convert non-numeric values to NaN
gender_df.replace('..', pd.NA, inplace=True)

start_column = 'A woman can apply for a passport in the same way as a man (1=yes; 0=no)'
selected_columns = gender_df.columns[gender_df.columns.get_loc(start_column):]


# Convert selected columns to numeric
gender_df[selected_columns] = gender_df[selected_columns].apply(pd.to_numeric, errors='coerce')

# Group by country and calculate the mean for each indicator
average_by_country = gender_df.groupby('country_name')[selected_columns].mean().reset_index()

In [27]:
df_filtered = average_by_country.dropna(axis=1, thresh=230)
df_filtered.to_csv('average_by_country.csv', index=False)



In [28]:
df_filtered.shape

(265, 268)